In [33]:
from unidecode import unidecode
import pandas as pd

In [35]:
article=pd.read_table('ref-extracts_june.csv',sep=',',error_bad_lines=False,encoding = "ISO-8859-1")
article['Author_list']=article.Author.map(lambda x: x.split(";"))

Authors=pd.DataFrame({'First_name':[],'Last_name':[],"Rank":[],"Title":[]})
for i in article.index.to_list():
    liste_authors=article.Author_list[i]
    n=0
    article_name=article.Title[i]
    #first author:
    first_author = liste_authors[0]
    first_name_1 = first_author.split(", ")[1].split(" ")[0]
    last_name_1 = first_author.split(",")[0]
    author=pd.DataFrame({'First_name':[first_name_1],'Last_name':[last_name_1],'Rank':['first'],'Title':[article_name]})
    Authors=pd.concat([Authors,author])
    
    #last author:
    last_author = liste_authors[-1]
    first_name_n = last_author.split(", ")[1].split(" ")[0]
    last_name_n = last_author.split(",")[0]
    author=pd.DataFrame({'First_name':[first_name_n],'Last_name':[last_name_n],'Rank':['last'],'Title':[article_name]})
    Authors=pd.concat([Authors,author])
        
        


C:\Users\CRI User\AppData\Local\Temp\ipykernel_13052\2599737196.py:1: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  article=pd.read_table('ref-extracts_june.csv',sep=',',error_bad_lines=False,encoding = "ISO-8859-1")


In [36]:
from unidecode import unidecode

Authors['First_name']=Authors.First_name.map(unidecode)
Authors['First_name']=Authors.First_name.map(lambda x: x.lower())
Authors['Last_name']=Authors.Last_name.map(unidecode)
Authors['Last_name']=Authors.Last_name.map(lambda x: x.lower())
Authors['First_name']=Authors.First_name.map(lambda x: x.replace(" ",""))
Authors['First_name']=Authors.First_name.map(lambda x: x.replace("-",""))
Authors=Authors.reset_index(drop=True)

In [37]:
name_gender=pd.read_table('name_gender_dataset.csv',sep=',')
name_gender['Name']=name_gender.Name.map(unidecode)
name_gender['Name']=name_gender.Name.map(lambda x: x.lower())
name_gender['Name']=name_gender.Name.map(lambda x: x.replace(" ",""))
name_gender['Name']=name_gender.Name.map(lambda x: x.replace("-",""))
name_gender=name_gender.pivot_table(index='Name',columns='Gender',values='Count',aggfunc=sum).fillna(0)
name_gender['Name']=name_gender.index
name_gender['Name']=name_gender.index
name_gender['Ratio_M']=name_gender.M/(name_gender.M+name_gender.F)
name_gender['Total_name']=name_gender.M+name_gender.F
name_gender=name_gender.loc[name_gender.Total_name>5]
name_gender['Gender']="U"
name_gender.loc[name_gender.Ratio_M>0.9,'Gender']='M'
name_gender.loc[name_gender.Ratio_M<0.10,'Gender']='F'
name_gender.reset_index(drop=True,inplace=True)

In [38]:
## Check if author in database
for i in Authors.index.to_list():
    Authors.loc[i,'isin_db']=len(name_gender.loc[name_gender.Name==Authors.First_name[i]])
Authors

,First_name,Last_name,Rank,Title,isin_db
0,gal,ofir,first,Contemporary Phage Biology: From Classic Model...,1.0
1,rotem,sorek,last,Contemporary Phage Biology: From Classic Model...,1.0
2,curtis,suttle,first,Marine viruses â major players in the global...,1.0
3,curtis,suttle,last,Marine viruses â major players in the global...,1.0
4,larissa,kever,first,Aminoglycoside Antibiotics Inhibit Phage Infec...,1.0
...,...,...,...,...,...
273,malcolm,white,last,An anti-CRISPR viral ring nuclease subverts ty...,1.0
274,erin,huiting,first,Bacteriophages inhibit and evade cGAS-like imm...,1.0
275,joseph,bondy-denomy,last,Bacteriophages inhibit and evade cGAS-like imm...,1.0
276,april,pawluk,first,"Anti-CRISPR: discovery, mechanism and function",1.0


In [14]:
Authors_unknown=Authors.loc[Authors.isin_db==0,]

Authors_known=Authors.loc[Authors.isin_db!=0,]
Authors_known=Authors_known.merge(name_gender[['Name','Gender']],left_on='First_name',right_on='Name')

In [58]:
pd.concat([Authors_known,Authors_unknown]).to_excel('All_authors_june23.xlsx')